In [2]:
#importing dependencies
import tensorflow as tf
import pandas as pd
import numpy as np

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils import shuffle
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [3]:
# Mounting google drive containing stackoverflow dataset
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Authenticate to gcloud account
from google.colab import auth
auth.authenticate_user()

In [0]:
# Reading data from the google drive
data = pd.read_csv( "/content/drive/My Drive/stackoverflow/stack.csv")


In [7]:
data.head()

,f0_,tags
0,problem with the size of the arrows on a vecto...,"python,matplotlib"
1,how to disable screen update in matplotlib i h...,"python,matplotlib,scipy"
2,wxpython gui: migrating gnuplot to matplotlib ...,"python,wxpython,matplotlib"
3,how to incorporate drag feature in wxpython i ...,"wxpython,wxwidgets,matplotlib"
4,matplotlib svg requires plugin to view i am tr...,"django,matplotlib"


In [0]:
# Getting Machine Learning tags in tag_split list
import re

def remove_unwanted_tags(string):
  lst = []        
  s = re.compile('(matplotlib|keras|pandas|scikit-learn)')
  lst = s.findall(string)
  return lst



In [0]:
l = len(data)
tag_split = []
for i in range(l):
  lst = remove_unwanted_tags(data['tags'].iloc[i])
  tag_split.append(lst)


Preprocessing Text



In [0]:
s = data['tags'].iloc[0].split(",")


In [0]:
# Replacing Tensorflow Keras Matplotlib Pandas Scikit-Learn
import re
l = len(data)
for i in range(l):
  s = data['f0_'].iloc[i]

  if re.search('matplotlib',s):
    data['f0_'].iloc[i] = re.sub("matplotlib","avocado",s) 

  if re.search('keras',s):
    data['f0_'].iloc[i] = re.sub("keras","avocado",s) 

  if re.search('scikit-learn',s):
    data['f0_'].iloc[i] = re.sub("scikit-learn","avocado",s)

  if re.search('pandas',s):
    data['f0_'].iloc[i] = re.sub("pandas","avocado",s)

  if re.search('tensorflow',s):
    data['f0_'].iloc[i] = re.sub("tensorflow","avocado",s) 
   

In [77]:
data = shuffle(data, random_state = 22)

,f0_,tags
78089,avocado get index of highest dot product i hav...,"python,numpy,pandas,dot-product"
209699,splitting avocado dataframe to feed into a sci...,"python,pandas,scikit-learn"
215804,how to freeze some layer after restore only pa...,tensorflow
144628,what is the cntk equivalent of a simple sgd on...,"tensorflow,cntk"
79460,add data to avocado dataframe using for loop w...,"python,pandas,dataframe,tweepy"


In [0]:
# Encoding 

tag_encoder = MultiLabelBinarizer()
tags_encoded = tag_encoder.fit_transform(tag_split)
num_tags = len(tags_encoded[0])


In [78]:
#Defining Training size and Test size

train_size = int(len(data)* 0.8)
print ("Train size %d"% train_size)
print("Test size %d"% (len(data)-train_size))


Train size 179922
Test size 44981


In [0]:
#Split tags
train_tags = tags_encoded[:train_size]
test_tags =  tags_encoded[train_size:]

In [80]:
train_tags

array([[0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       ...,
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0]])

In [81]:
# Pre-processing data: create our tokenizer class
%%writefile preprocess.py

from tensorflow.keras.preprocessing import text

class TextPreprocessor(object):
  def __init__(self, vocab_size):
    self._vocab_size = vocab_size
    self._tokenizer = None
  
  def create_tokenizer(self, text_list):
    tokenizer = text.Tokenizer(num_words=self._vocab_size)
    tokenizer.fit_on_texts(text_list)
    self._tokenizer = tokenizer

  def transform_text(self, text_list):
    text_matrix = self._tokenizer.texts_to_matrix(text_list)
    return text_matrix


Writing preprocess.py


In [82]:
!ls

adc.json  drive  preprocess.py	sample_data


In [0]:
# Create vocab from training corpus
from preprocess import TextPreprocessor

VOCAB_SIZE=400 # This is a hyperparameter, try out different values for your dataset

train_qs = data['f0_'].values[:train_size]
test_qs = data['f0_'].values[train_size:]

processor = TextPreprocessor(VOCAB_SIZE)
processor.create_tokenizer(train_qs)

body_train = processor.transform_text(train_qs)
body_test = processor.transform_text(test_qs)

In [85]:
print(len(body_train[0]))
print(body_train[0])

400
[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1.
 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1.
 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1.
 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0.
 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [0]:
# Save the processor state of the tokenizer
import pickle

with open('./processor_state.pkl', 'wb') as f:
  pickle.dump(processor, f)

In [0]:
#Creating Model
def create_model(vocab_size, num_tags):
  
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Dense(50, input_shape=(VOCAB_SIZE,), activation='relu'))
  model.add(tf.keras.layers.Dense(25, activation='relu'))
  model.add(tf.keras.layers.Dense(num_tags, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model


In [90]:
my_model = create_model(VOCAB_SIZE,num_tags)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [91]:
#Training Model
my_model.fit(body_train,train_tags,epochs= 4,batch_size = 128, validation_split=0.1)

Train on 161929 samples, validate on 17993 samples
Epoch 1/4
161929/161929 [==============================] - 4s 23us/sample - loss: 0.4368 - acc: 0.7914 - val_loss: 0.4325 - val_acc: 0.7945
Epoch 2/4
161929/161929 [==============================] - 4s 22us/sample - loss: 0.4329 - acc: 0.7938 - val_loss: 0.4329 - val_acc: 0.7930
Epoch 3/4
161929/161929 [==============================] - 3s 21us/sample - loss: 0.4317 - acc: 0.7946 - val_loss: 0.4326 - val_acc: 0.7948
Epoch 4/4
161929/161929 [==============================] - 4s 22us/sample - loss: 0.4304 - acc: 0.7958 - val_loss: 0.4333 - val_acc: 0.7909


In [94]:
#Evaluate
my_model.evaluate(body_test,test_tags,batch_size = 128)

#Saving model
my_model.save('keras_saved.h5')

44981/44981 [==============================] - 0s 9us/sample - loss: 0.4308 - acc: 0.7954


In [95]:
#Testing Model Locally


# Use custom model prediction to save our model + tokenizer
%%writefile model_prediction.py
import pickle
import os
import numpy as np

class CustomModelPrediction(object):

  def __init__(self, model, processor):
    self._model = model
    self._processor = processor
  
  def predict(self, instances, **kwargs):
    preprocessed_data = self._processor.transform_text(instances)
    predictions = self._model.predict(preprocessed_data)
    return predictions.tolist()

  @classmethod
  def from_path(cls, model_dir):
    import tensorflow.keras as keras
    model = keras.models.load_model(
      os.path.join(model_dir,'keras_saved_model.h5'))
    with open(os.path.join(model_dir, 'processor_state.pkl'), 'rb') as f:
      processor = pickle.load(f)

    return cls(model, processor)

Writing model_prediction.py


In [0]:
sample_question = [
  "How to preprocess strings in Keras models Lambda layer? I have the problem that the value passed on to the Lambda layer (at compile time) is a placeholder generated by keras (without values). When the model is compiled, the .eval () method throws the error: You must feed a value for placeholder tensor 'input_1' with dtype string and shape [?, 1] def text_preprocess(x): strings = tf.keras.backend.eval(x) vectors = [] for string in strings: vector = string_to_one_hot(string.decode('utf-8')) vectors.append(vector) vectorTensor = tf.constant(np.array(vectors),dtype=tf.float32) return vectorTensor input_text = Input(shape=(1,), dtype=tf.string) embedding = Lambda(text_preprocess)(input_text) dense = Dense(256, activation='relu')(embedding) outputs = Dense(2, activation='softmax')(dense) model = Model(inputs=[input_text], outputs=outputs) model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy']) model.summary() model.save('test.h5') If I pass a string array into the input layer statically, I can compile the model, but I get the same error if I want to convert the model to tflite. #I replaced this line: input_text = Input(shape=(1,), dtype=tf.string) #by this lines: test = tf.constant(['Hello', 'World']) input_text = Input(shape=(1,), dtype=tf.string, tensor=test) #but calling this ... converter = TFLiteConverter.from_keras_model_file('string_test.h5') tfmodel = converter.convert() #... still leads to this error: InvalidArgumentError: You must feed a value for placeholder tensor 'input_3' with dtype string and shape [2] [[{{node input_3}}]] ",
  "Change the bar item name in Pandas I have a test excel file like: df = pd.DataFrame({'name':list('abcdefg'), 'age':[10,20,5,23,58,4,6]}) print (df) name  age 0    a   10 1    b   20 2    c    5 3    d   23 4    e   58 5    f    4 6    g    6 I use Pandas and matplotlib to read and plot it: import pandas as pd import numpy as np import matplotlib.pyplot as plt import os excel_file = 'test.xlsx' df = pd.read_excel(excel_file, sheet_name=0) df.plot(kind='bar') plt.show() the result shows: enter image description here it use index number as item name, how can I change it to the name, which stored in column name?"
]